In [1]:
import os

In [2]:
%pwd

'C:\\Users\\Arghya\\OneDrive\\Documents\\KIDNEY-Disease-Classification\\research'

In [3]:
os.chdir("../")
%pwd

'C:\\Users\\Arghya\\OneDrive\\Documents\\KIDNEY-Disease-Classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    local_source_path: Path

In [5]:
import unittest

# Fixes the incompatibility between 'ensure' and Python 3.13 interpreter
if not hasattr(unittest.TestCase, 'assertRaisesRegexp'):
    unittest.TestCase.assertRaisesRegexp = unittest.TestCase.assertRaisesRegex

print("Patch applied successfully!")

Patch applied successfully!


In [6]:
from source.CNN_Disease_Classifier.constants import *
from source.CNN_Disease_Classifier.utils.common import  read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            local_source_path=config.local_source_path
        )

        return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
from source.CNN_Disease_Classifier import logger
from source.CNN_Disease_Classifier.utils.common import get_size

Defining the Data Ingestion class and Updating the components

In [9]:
import os
import shutil
import gdown
import zipfile
from pathlib import Path

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """
        Attempts to fetch data from a local path first,
        falling back to the URL if the local path is unavailable.
        """
        try:
            # 1. Define the destination for the file to end up (artifacts folder)
            zip_download_dir = self.config.local_data_file
            os.makedirs(os.path.dirname(zip_download_dir), exist_ok=True)

            # 2. TRY: Check if the local_source_path exists and is a file
            # Assuming 'local_source_path' is added to your Config
            local_path = Path(self.config.local_source_path)

            if not local_path.exists():
                raise FileNotFoundError("Local file not found, switching to download...")

            logger.info(f"Local file found at {local_path}. Copying to {zip_download_dir}...")
            shutil.copy(local_path, zip_download_dir)
            logger.info(f"Successfully ingested data from local source.")

        except Exception as e:
            # 3. EXCEPT: Fallback to internet download
            logger.warning(f"Could not ingest locally due to: {e}")

            dataset_url = self.config.source_URL
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            try:
                # Extracting ID from Google Drive URL
                file_id = dataset_url.split("/")[-2]
                prefix = 'https://drive.google.com/uc?/export=download&id='
                gdown.download(prefix + file_id, zip_download_dir)
                logger.info(f"Downloaded data from internet into {zip_download_dir}")
            except Exception as download_error:
                logger.error(f"Failed to download from internet: {download_error}")
                raise download_error

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted zip file to {unzip_path}")

Updating the Pipeline

In [10]:
# Ensuring that YAML file is not empty
import yaml

file_path = "C:/Users/Arghya/OneDrive/Documents/KIDNEY-Disease-Classification/config/config.yaml" # Adjust this to your actual path
with open(file_path, "r") as f:
    content = yaml.safe_load(f)
    print(content)

{'artifacts_root': 'artifacts', 'data_ingestion': {'local_source_path': 'C:/Users/Arghya/OneDrive/Documents/KIDNEY-Disease-Classification/research/archive.zip', 'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.h5'}}


In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-12-26 23:53:15,150: INFO: common: yaml file: C:\Users\Arghya\OneDrive\Documents\KIDNEY-Disease-Classification\config\config.yaml loaded successfully]
[2025-12-26 23:53:15,188: INFO: common: yaml file: C:\Users\Arghya\OneDrive\Documents\KIDNEY-Disease-Classification\params.yaml loaded successfully]
[2025-12-26 23:53:15,216: INFO: common: created directory at: artifacts]
[2025-12-26 23:53:15,218: INFO: common: created directory at: artifacts/data_ingestion]
[2025-12-26 23:53:15,296: INFO: 1530172710: Local file found at C:\Users\Arghya\OneDrive\Documents\KIDNEY-Disease-Classification\research\archive.zip. Copying to artifacts/data_ingestion/data.zip...]
[2025-12-26 23:53:19,692: INFO: 1530172710: Successfully ingested data from local source.]
[2025-12-26 23:53:49,635: INFO: 1530172710: Extracted zip file to artifacts/data_ingestion]
